In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import random 
import cv2
import os
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, MaxPool2D, Conv2D, Input, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Set the path to the dataset
input_path = r'C:\Users\wmonsri\Desktop\chest_xray'
model_location = r'C:\Users\wmonsri\Desktop\xray\best_model.h5'

fig, ax = plt.subplots(2, 3, figsize=(15, 7))  
ax = ax.ravel()  
plt.tight_layout()
for i, _set in enumerate(['train', 'val', 'test']):
    set_path = os.path.join(input_path, _set)
    ax[i].imshow(plt.imread(os.path.join(set_path, 'NORMAL', os.listdir(os.path.join(set_path, 'NORMAL'))[0])), cmap='gray')
    ax[i].set_title(f'set{_set}, Condition: NORMAL')
    ax[i+3].imshow(plt.imread(os.path.join(set_path, 'PNEUMONIA', os.listdir(os.path.join(set_path, 'PNEUMONIA'))[0])), cmap='gray')
    ax[i+3].set_title(f'set{_set}, Condition: PNEUMONIA')
plt.show()

for _set in ['train', 'val', 'test']:
    print(os.path.join(input_path, _set))
    n_normal = len(os.listdir(os.path.join(input_path, _set, 'NORMAL')))
    n_pneumonia = len(os.listdir(os.path.join(input_path, _set, 'PNEUMONIA')))
    print(f'{_set}, normal images {n_normal}, pneumonia images {n_pneumonia}')


def process_data(img_dim, batch_size):
    train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, vertical_flip=True)
    test_val_datagen = ImageDataGenerator(rescale=1./255)
    
    train_gen = train_datagen.flow_from_directory(
        directory=os.path.join(input_path, 'train'),
        target_size=(img_dim, img_dim),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )
    
    test_gen = test_val_datagen.flow_from_directory(
        directory=os.path.join(input_path, 'test'),
        target_size=(img_dim, img_dim),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )
    
    test_data = []
    test_labels = []
    for cond in ['NORMAL', 'PNEUMONIA']:
        for img in os.listdir(os.path.join(input_path, 'test', cond)):
            img_path = os.path.join(input_path, 'test', cond, img)
            img = plt.imread(img_path)
            img = cv2.resize(img, (img_dim, img_dim))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            label = 0 if cond == 'NORMAL' else 1
            test_data.append(img)
            test_labels.append(label)
        
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    return train_gen, test_gen, test_data, test_labels

# Set image dimensions, epochs, and batch size
img_dims = 150
epochs = 10
batch_size = 32


train_gen, test_gen, test_data, test_labels = process_data(img_dims, batch_size)


if os.path.exists(model_location):
    model = load_model(model_location)
    print("Loaded model from disk")
else:
    inputs = Input(shape=(img_dims, img_dims, 3))
    X = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    X = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(X)
    X = MaxPool2D(pool_size=(2, 2))(X)
    
    # Add more layers as needed

    X = Flatten()(X)
    X = Dense(units=1, activation='sigmoid')(X)  # Ensure only one unit in the output layer for binary classification

    # Create the model
    model = Model(inputs=inputs, outputs=X)

    
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint(filepath=model_location, save_best_only=True, save_weights_only=False)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min') #we should remove the early stopping and test

# Train the model
history = model.fit_generator(
    train_gen, 
    steps_per_epoch=train_gen.samples // batch_size,
    epochs=epochs, 
    validation_data=test_gen, 
    validation_steps=test_gen.samples // batch_size, 
    callbacks=[checkpoint, lr_reduce, early_stop]
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from keras.models import load_model
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain

# Set the path to the dataset
input_path = r'C:\Users\wmonsri\Desktop\chest_xray\test'

# Set the location to load the model
model_location = r'C:\Users\wmonsri\Desktop\xray\best_model.h5'

# Load the model
if os.path.exists(model_location):
    model = load_model(model_location)
    print("Loaded CNN model from disk")
else:
    print("CNN Model not found. Please train the model first.")

# Function to process test data with NLP analysis
def process_test_data_with_nlp(img_dim):
    test_data = []
    image_paths = []
    for class_folder in ['NORMAL', 'PNEUMONIA']:
        class_path = os.path.join(input_path, class_folder)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = cv2.resize(cv2.imread(img_path, cv2.IMREAD_COLOR), (img_dim, img_dim))
            test_data.append(img)
            image_paths.append(img_path)
    return test_data, image_paths

def contour_lungs(img):
    # Convert the image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred_img = cv2.GaussianBlur(gray_img, (5, 5), 0)

    # Apply adaptive thresholding to segment the lungs
    _, threshold_img = cv2.threshold(blurred_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(threshold_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask to draw the contours
    contour_mask = np.zeros_like(img)

    # Draw contours on the mask
    cv2.drawContours(contour_mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Convert the mask to grayscale
    contour_mask_gray = cv2.cvtColor(contour_mask, cv2.COLOR_BGR2GRAY)

    # Calculate the opacity degree based on the contour area
    total_area = img.shape[0] * img.shape[1]
    highlighted_area = cv2.countNonZero(contour_mask_gray)
    opacity_degree = (highlighted_area / total_area) * 100  # Opacity as a percentage

    # Apply the mask to the original image to get the contoured image
    contoured_img = cv2.bitwise_and(img, img, mask=contour_mask_gray)

    return contoured_img, opacity_degree


 

def generate_nlp_interpretation(img_name, probability, opacity_degree):
    # Generate textual summary for the finding
    text = f"Probability of Pneumonia: {probability}\nOpacity Degree of Lungs: {opacity_degree:.2f}%"
    return text

# Initialize LLM model
llm = CTransformers(model="TheBloke/Llama-2-13B-Ensemble-v5-GGUF", model_file="llama-2-13b-ensemble-v5.Q2_K.gguf", model_type="llama")

# Define template
template = """
              Describe the findings in the following image: `{image_name}`.
              Condition: Pneumonia
              Probability of Pneumonia (CNN Model): {probability}
              Opacity Degree of Lungs: {opacity_degree:.2f}%
              Return your response in bullet points which covers the key points of the findings.
              ```{text}```
              interpret these results to confirm or deny if it is a case of pneumonia.
              BULLET POINT SUMMARY:
              - Condition: Pneumonia
              - Probability of Pneumonia (CNN Model): {probability}
              - Opacity Degree of Lungs: {opacity_degree:.2f}%
           """


prompt = PromptTemplate(template=template, input_variables=["image_name", "probability", "opacity_degree", "text"])

# Initialize LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Set image dimensions
img_dims = 150

# Process the test data and generate NLP interpretations
test_data, image_paths = process_test_data_with_nlp(img_dims)

# Check if test_data is empty
if not test_data:
    print("No test data found. Please check your data loading logic.")
else:
    # Analyze images and generate interpretations
    for img, img_path in zip(test_data, image_paths):
        img_name = os.path.basename(img_path)
        print(f"Image Name: {img_name}")
        
        # Predict using pre-trained CNN model
        probability = model.predict(np.array([img]))[0][0]
        
        # Contour lungs and calculate opacity degree
        contoured_img, opacity_degree = contour_lungs(img)
        
        # Display the original and contoured images side by side
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axes[0].set_title("Original Image")
        axes[0].axis('off')
        
        # Highlight opacity in red on the contoured image
        contoured_with_opacity = np.copy(contoured_img)
        contoured_with_opacity[:, :, 0] = np.where(contoured_with_opacity[:, :, 0] == 0, 0, 255)
        axes[1].imshow(cv2.cvtColor(contoured_with_opacity, cv2.COLOR_BGR2RGB))
        axes[1].set_title("Contoured Lungs (Opacity Highlighted)")
        axes[1].axis('off')
        
        plt.show()
        
        # Print opacity degree, probability, and image name
        print(f"Opacity Degree of Lungs: {opacity_degree:.2f}%")
        print(f"Probability of Pneumonia: {probability}")
        
        # Generate NLP interpretation
        nlp_interpretation = llm_chain.run({'image_name': img_name, 'probability': probability, 'opacity_degree': opacity_degree, 'text': ''})

        print("Findings Summary:")
        print(nlp_interpretation)


In [ ]:
from flask import Flask, request, jsonify, render_template
from werkzeug.utils import secure_filename
from keras.models import load_model
import numpy as np
import cv2
import os
import logging
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain

# Configure logging
logging.basicConfig(level=logging.DEBUG)

app = Flask(__name__, template_folder=r'C:\Users\wmonsri\Desktop\templates')
app.config['UPLOAD_FOLDER'] = 'uploads'

# Set the location to load the CNN model
cnn_model_location = 'C:\\Users\\wmonsri\\Desktop\\xray\\best_model.h5'
if os.path.exists(cnn_model_location):
    cnn_model = load_model(cnn_model_location)
    logging.info("Loaded CNN model from disk")
else:
    logging.error("CNN Model not found. Please train the model first.")

# Initialize LLM model
llm = CTransformers(model="TheBloke/Llama-2-13B-Ensemble-v5-GGUF",
                    model_file="llama-2-13b-ensemble-v5.Q2_K.gguf",
                    model_type="llama")

# Define template
template = """
              Describe the findings in the following image: `{image_name}`.
              Condition: Pneumonia
              Probability of Pneumonia (CNN Model): {probability}
              Opacity Degree of Lungs: {opacity_degree:.2f}%
              Return your response in bullet points which covers the key points of the findings.
              ```{text}```
              interpret these results to confirm or deny if it is a case of pneumonia.
              BULLET POINT SUMMARY:
              - Condition: Pneumonia
              - Probability of Pneumonia (CNN Model): {probability}
              - Opacity Degree of Lungs: {opacity_degree:.2f}%
           """

# Initialize LLMChain
prompt = PromptTemplate(template=template, input_variables=["image_name", "probability", "opacity_degree", "text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)


# Function to contour lungs and calculate opacity degree
def contour_lungs(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred_img = cv2.GaussianBlur(gray_img, (5, 5), 0)
    _, threshold_img = cv2.threshold(blurred_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(threshold_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_mask = np.zeros_like(img)
    cv2.drawContours(contour_mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)
    contour_mask_gray = cv2.cvtColor(contour_mask, cv2.COLOR_BGR2GRAY)
    total_area = img.shape[0] * img.shape[1]
    highlighted_area = cv2.countNonZero(contour_mask_gray)
    opacity_degree = (highlighted_area / total_area) * 100
    contoured_img = cv2.bitwise_and(img, img, mask=contour_mask_gray)
    contoured_img[:, :, 0] = np.where(contour_mask_gray == 255, 255, contoured_img[:, :, 0])  # Highlight opacity in blue color
    contoured_img[:, :, 1] = np.where(contour_mask_gray == 255, 0, contoured_img[:, :, 1])
    contoured_img[:, :, 2] = np.where(contour_mask_gray == 255, 0, contoured_img[:, :, 2])
    return contoured_img, opacity_degree


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if 'file' not in request.files:
            return jsonify({'error': 'No file part'})

        file = request.files['file']
        if file.filename == '':
            return jsonify({'error': 'No selected file'})

        # Read the uploaded image
        image = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
        contoured_img, opacity_degree = contour_lungs(image)

        # Resize the processed image to match the input shape of the model (if necessary)
        resized_img = cv2.resize(contoured_img, (150, 150))

        # Predict using the pre-trained CNN model
        probability = cnn_model.predict(np.expand_dims(resized_img, axis=0))[0][0]

        # Convert probability to Python float
        probability = float(probability)

        # Generate NLP interpretation
        nlp_interpretation = llm_chain.run(
            {'image_name': file.filename, 'probability': probability, 'opacity_degree': opacity_degree, 'text': ''})

        # Debugging: Print processed image path
        processed_img_name = 'processed_' + secure_filename(file.filename)
        processed_img_path = os.path.join('C:\\Users\\wmonsri\\Desktop\\process', processed_img_name)
        print("Processed image path:", processed_img_path)

        # Save the processed image
        cv2.imwrite(processed_img_path, contoured_img)

        return jsonify({'processedImagePath': processed_img_path,
                        'probability': probability,
                        'opacityPercentage': opacity_degree,
                        'nlpInterpretation': nlp_interpretation})

    return render_template('index.html')



if __name__ == '__main__':
    app.run(debug=False)


DEBUG:h5py._conv:Creating converter from 3 to 5



INFO:root:Loaded CNN model from disk
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/TheBloke/Llama-2-13B-Ensemble-v5-GGUF/revision/main HTTP/1.1" 200 1742


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/TheBloke/Llama-2-13B-Ensemble-v5-GGUF/revision/main HTTP/1.1" 200 1742


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

...

1/1 [==============================] - 0s 323ms/step


C:\Users\wmonsri\Anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:24:25] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:24:27] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:24:30] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:24:31] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:24:34] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:24:36] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:24:38] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:24:40] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:24:42] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:

Processed image path: C:\Users\wmonsri\Desktop\process\processed_IM-0122-0001.jpeg


INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:31] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:33] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:35] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:37] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:39] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:41] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:43] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:45] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:47] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:49] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:51] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:27:53] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024

INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:37] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:39] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:41] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:43] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:46] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:48] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:50] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:52] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:54] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:56] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:30:58] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 14:31:00] "GET /update HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024